In [2]:
import MLC.mlc as mlc
import numpy as np
import torch
import torch.nn as nn
import MLC.demo.model.resnet as Resnet

In [3]:
# resnet = Resnet.resnet18(num_classes=10)
# img_np = np.random.rand(1, 1, 384, 384).astype('float32')


In [14]:

class resnet(nn.Module):
    def __init__(self):
        super().__init__()
        self.maxpool = nn.MaxPool2d((3, 3), stride=1)
        self.conv1 = nn.Conv2d(1, 1, 3, 1)
        self.bn = nn.BatchNorm2d(1)
        self.relu = nn.ReLU()
        self.avg = nn.AdaptiveAvgPool2d((1,1))
    
    def forward(self, x):
        x = x+ x
        # x = self.maxpool(x)
        x = self.conv1(x)
        # x = self.bn(x)
        x = self.relu(x)
        # x = self.avg(x)
        x = torch.flatten(x, 1)
        return x

In [15]:
fx_module = torch.fx.symbolic_trace(resnet())
fx_module.graph.print_tabular()
for i in fx_module.graph.nodes:
    print(i, i.target, type(i.target))

type(abs)

opcode         name     target                                                      args        kwargs
-------------  -------  ----------------------------------------------------------  ----------  --------
placeholder    x        x                                                           ()          {}
call_function  add      <built-in function add>                                     (x, x)      {}
call_module    conv1    conv1                                                       (add,)      {}
call_module    relu     relu                                                        (conv1,)    {}
call_module    avg      avg                                                         (relu,)     {}
call_function  flatten  <built-in method flatten of type object at 0x7f7363859ec0>  (avg, 1)    {}
output         output   output                                                      (flatten,)  {}
x x <class 'str'>
add <built-in function add> <class 'builtin_function_or_method'>
conv1 conv1 <cla

builtin_function_or_method

In [16]:
resnet_fx_module = mlc.from_fx(fx_module, [(1, 1, 384, 384)])
resnet_fx_module.show()

/staff/qiaoliang/anaconda3/envs/MLC/lib/python3.8/site-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/staff/qiaoliang/anaconda3/envs/MLC/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


In [17]:
resnet_fused = mlc.FuseDenseAddPass()(resnet_fx_module)
resnet_fused.show()

/staff/qiaoliang/anaconda3/envs/MLC/lib/python3.8/site-packages/tvm/script/highlight.py:117: UserWarning: No module named 'black'
To print formatted TVM script, please install the formatter 'Black':
/staff/qiaoliang/anaconda3/envs/MLC/bin/python -m pip install "black==22.3.0" --upgrade --user
  warnings.warn(


In [18]:
lowresnet = mlc.LowerToTensorIRPass()(resnet_fused)
lowresnet.show()

TVMError: Traceback (most recent call last):
  5: TVMFuncCall
  4: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::TypedPackedFunc<tvm::IRModule (tvm::transform::Pass, tvm::IRModule)>::AssignTypedLambda<tvm::transform::{lambda(tvm::transform::Pass, tvm::IRModule)#7}>(tvm::transform::{lambda(tvm::transform::Pass, tvm::IRModule)#7}, std::string)::{lambda(tvm::runtime::TVMArgs const&, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  3: tvm::transform::Pass::operator()(tvm::IRModule) const
  2: tvm::transform::Pass::operator()(tvm::IRModule, tvm::transform::PassContext const&) const
  1: tvm::transform::ModulePassNode::operator()(tvm::IRModule, tvm::transform::PassContext const&) const
  0: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<TVMFuncCreateFromCFunc::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#2}> >::Call(tvm::runtime::PackedFun

In [ ]:
import tvm
tvm.ir.Op.get('relax.nn.adaptive_avg_pool2d')

Op(relax.nn.adaptive_avg_pool2d)